In [37]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

import datetime
from datetime import timedelta
# !pip install holidays
import holidays

os.chdir("/content/drive/MyDrive/3. Grad School/LG Aimers")

In [ ]:
# 데이터 불러오기
data = pd.read_csv("DATA/train/train.csv")

#### 파생변수 생성

In [38]:
# 월(month) -> 계절 매핑 딕셔너리
month_to_season = {
    1: "Winter", 2: "Winter", 12: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"}

In [39]:
# 월별 가중치 매핑
# monthly_weights = {
#     1: 1.5, 2: 1.5, 3: 0.5,
#     4: 0.8, 5: 0.8, 6: 0.8,
#     7: 1.2, 8: 1.2, 9: 0.8,
#     10: 0.8, 11: 0.8, 12: 1.2}

monthly_weights = {
    1: 2.2, 2: 1.8, 3: 0.3,
    4: 1.01, 5: 0.7, 6: 0.8,
    7: 0.5, 8: 0.5, 9: 0.8,
    10: 1.55, 11: 1.03, 12: 1.4}

In [40]:
# 요일별 가중치 매핑
# weekly_weights = {
#     "Monday": 0.8, "Tuesday": 0.8, "Wednesday": 0.8,
#     "Thursday": 0.8, "Friday": 1.2, "Saturday": 1.7,
#     "Sunday": 1.5}

weekly_weights = {
    "Monday": 0.78, "Tuesday": 0.85, "Wednesday": 0.81,
    "Thursday": 9.9, "Friday": 1.2, "Saturday": 1.53,
    "Sunday": 1.3}

In [41]:
class Make_Variables():
        def __init__(self, data = None, date = None, predict = 7, month_to_season = None, monthly_weights = None, weekly_weights = None):
            self.data = data
            self.date = date
            self.predict = predict
            self.month_to_season = month_to_season
            self.monthly_weights = monthly_weights
            self.weekly_weights = weekly_weights

        def update_kor_holidays(self):
            """국경일 추가"""
            kor_holidays = holidays.KR(years = [2023, 2024, 2025])
            kor_holidays.update({
                # datetime.date(2023,2,14) : "Valentine's Day",
                # datetime.date(2023,3,14) : "White Day",
                # datetime.date(2023,11,11) : "Pepero Day",
                # datetime.date(2024,2,14) : "Valentine's Day",
                # datetime.date(2024,3,14) : "White Day",
                # datetime.date(2024,11,11) : "Pepero Day",
                # datetime.date(2025,2,14) : "Valentine's Day",
                # datetime.date(2025,3,14) : "White Day",
                # datetime.date(2025,11,11) : "Pepero Day",

                datetime.date(2024,10,1) : "Temporary Holiday", # 국군의 날 임시공휴일
                datetime.date(2025,1,27) : "Temporary Holiday", # 설날 임시공휴일
                datetime.date(2025,3,3) : "Temporary Holiday", # 삼일절 대체공휴일
                datetime.date(2025, 5, 29) : "Election Period",
                datetime.date(2025, 5, 30) : "Election Period",
                datetime.date(2025, 6, 3) : "Presidential Election Day"})
            return kor_holidays

        def check_holidays(self, date, kor_holidays) -> int:
            """날짜 받아서 공휴일/주말 여부 출력"""
            # date = pd.Timestamp(date)
            if isinstance(date, pd.Series):
                check_holiday = date.dt.date.isin(kor_holidays)
                check_weekend = date.dt.weekday >= 5
            else:
                check_holiday = date.date() in kor_holidays
                check_weekend = date.weekday() >= 5
            is_holiday = (check_holiday | check_weekend)
            return is_holiday

        def get_sandwich_score(self, data, is_holiday_col) -> pd.DataFrame:
            """데이터프레임 기준으로 샌드위치 점수 계산"""
            data = data.reset_index(drop = True)
            data['is_sandwich'] = 0
            is_holiday = data[is_holiday_col].astype(int)
            for idx in range(len(data)):
                if idx == 0 or idx == len(data) - 1: # 첫날, 마지막 날
                    continue

                # 앞/뒤 하루씩 봤을 때 모두 휴일 -> 5점
                if (is_holiday.iloc[idx - 1] == 1) and (is_holiday.iloc[idx + 1] == 1): # 하루 전이랑 다음 날이 공휴일이면
                    data.iloc[idx, data.columns.get_loc('is_sandwich')] = 5

                # 앞/뒤 이틀씩 봤을 때 휴일 3일 -> 3점, 2일 -> 2점
                elif idx > 1 and idx < len(is_holiday) - 2: # 셋째날, 마지막에서 세 번째 날
                    start_idx = idx - 2
                    end_idx = idx + 2
                    nearby_holidays = (is_holiday.iloc[start_idx : end_idx + 1].sum() - is_holiday.iloc[idx])
                    if nearby_holidays == 3:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 3
                    elif nearby_holidays == 2:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 2
                    else:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 0
            return data

        def get_sandwich_score_for_dates(self, date, kor_holidays) -> int:
            """특정 날짜를 받아와서 앞뒤 날짜를 구하고, 샌드위치 점수 계산"""
            # 하루씩
            prev_date, next_date = date - timedelta(days = 1), date + timedelta(days = 1)
            prev_hol, next_hol = self.check_holidays(prev_date, kor_holidays), self.check_holidays(next_date, kor_holidays) # T/F Bool
            if prev_hol and next_hol: # 바로 다음 날들이 휴일이라면
                return 3
            days_offsets = [-2, -1, 1, 2] # 앞뒤로 이틀 살펴보기
            nearby_holidays = sum(self.check_holidays(date + timedelta(days = d), kor_holidays) for d in days_offsets)
            if nearby_holidays == 3: # 앞뒤 4일 중에 3일이 휴일이면
                return 2
            elif nearby_holidays == 2: # 앞뒤 4일 중에 2일이 휴일이면
                return 1
            else:
                return 0

        def get_month_weights(self, data = None, monthly_weights = monthly_weights):
            """월별 가중치 부여"""
            # 데이터프레임 들어오면
            if data is not None:
                data['month_weight'] = data['month'].map(monthly_weights)
                return data

        def get_week_weights(self, data = None, weekly_weights = weekly_weights):
            """요일별 가중치 부여"""
            # 데이터프레임 들어오면
            if data is not None:
                data['week_weight'] = data['weekday'].map(weekly_weights)
                return data

        def get_prev_days(self, data, test_df = None, date = None, menu = None, howmany = 7):
            """
            일요일 날짜 받아와서 직전 주차의 일-토 매출수량 평균 계산
            주의 - test data에서 생성할 때는 참고할 데이터와 붙여넣을 데이터가 다름
            data : 참고할 데이터
            test_df : 참고할 데이터
            """
            if test_df is None:
                # 혹시 모르니까 검증
                if date.weekday() == 6:
                    # 이전 날짜들
                    prev_start = date - timedelta(days = howmany)
                    prev_end = date - timedelta(days = 1)
                    prev_data = data[(data['영업일자'] >= prev_start) & (data['영업일자'] <= prev_end) & (data['영업장명_메뉴명'] == menu)]
                    prev_avg = prev_data['매출수량'].mean()
                    prev_sd = prev_data['매출수량'].std()
                    # 첫 주 0으로 처리
                    if pd.isna(prev_avg):
                        prev_avg = 0
                    if pd.isna(prev_sd):
                        prev_sd = 0
                    week_end = date + timedelta(days = 6)
                    curr_mask = (data['영업일자'] >= date) & (data['영업일자'] <= week_end) & (data['영업장명_메뉴명'] == menu)
                    colname_mean = f"prev_avg_{howmany}"
                    colname_sd = f"prev_sd_{howmany}"
                    data.loc[curr_mask, colname_mean] = prev_avg
                    data.loc[curr_mask, colname_sd] = prev_sd
                    return data
                else:
                    return np.nan

            # test data라면
            else:
                # 혹시 모르니까 검증
                if date.weekday() == 6:
                    # 이전 날짜들
                    prev_start = date - timedelta(days = howmany)
                    prev_end = date - timedelta(days = 1)
                    prev_data = test_df[(test_df['영업일자'] >= prev_start) & (test_df['영업일자'] <= prev_end) & (test_df['영업장명_메뉴명'] == menu)]
                    prev_avg = prev_data['매출수량'].mean()
                    prev_sd = prev_data['매출수량'].std()
                    # 첫 주 0으로 처리
                    if pd.isna(prev_avg):
                        prev_avg = 0
                    if pd.isna(prev_sd):
                        prev_sd = 0
                    week_end = date + timedelta(days = 6)
                    curr_mask = (data['영업일자'] >= date) & (data['영업일자'] <= week_end) & (data['영업장명_메뉴명'] == menu)
                    colname_mean = f"prev_avg_{howmany}"
                    colname_sd = f"prev_sd_{howmany}"
                    data.loc[curr_mask, colname_mean] = prev_avg
                    data.loc[curr_mask, colname_sd] = prev_sd
                    return data
                else:
                    return np.nan

        #########################################################################
        def add_ohe_cols(self, data):
            season_order = ["Spring", "Summer", "Autumn", "Winter"]
            month_order = list(range(1,13))
            weekday_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

            data["season"] = pd.Categorical(data["season"], categories=season_order)
            data["month"]   = pd.Categorical(data["month"].astype(int), categories = month_order)
            data["weekday"] = pd.Categorical(data["weekday"], categories = weekday_order)

            season_ohe = pd.get_dummies(data["season"], prefix="season", dtype="int8")
            m_ohe  = pd.get_dummies(data["month"],   prefix="m",  dtype="int8")
            wd_ohe = pd.get_dummies(data["weekday"], prefix="wd", dtype="int8")

            return pd.concat([data, season_ohe, m_ohe, wd_ohe], axis=1)
        #########################################################################

        # train, test 공통
        def make_fund_variables(self, data, month_to_season = month_to_season):
            # 영업일자 -> datetime
            data['영업일자'] = pd.to_datetime(data['영업일자'])

            # 연, 월, 일, 요일 분리
            data['year'] = data['영업일자'].dt.year
            data['month'] = data['영업일자'].dt.month
            data['day'] = data['영업일자'].dt.day
            data['weekday'] = data['영업일자'].dt.day_name()
            data['weekday_enc'] = data['영업일자'].dt.weekday

            # 계절 변수 생성
            data['season'] = data['month'].map(month_to_season)

            # 연도 차이 변수 생성
            data['year_enc'] = data['year'] - 2023

            # 월, 일, 요일 사이클릭 변환
            data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
            data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

            data['day_sin'] = np.sin(2 * np.pi * data['day'] / 31)
            data['day_cos'] = np.cos(2 * np.pi * data['day'] / 31)

            data['weekday_sin'] = np.sin(2 * np.pi * data['weekday_enc'] / 7)
            data['weekday_cos'] = np.cos(2 * np.pi * data['weekday_enc'] / 7)

            # 공휴일 확인
            kor_holidays = self.update_kor_holidays()
            check_holiday = data['영업일자'].dt.date.isin(kor_holidays)
            check_weekend = data['weekday'].isin(['Saturday', 'Sunday'])
            data['is_holiday'] = (check_holiday | check_weekend).astype(int) # 공휴일 + 주말
            data['holiday_name'] = data['영업일자'].dt.date.map(kor_holidays)

            # one-hot
            data = self.add_ohe_cols(data)

            return data

        # train의 입력 데이터
        def make_variables_train(self, data):
            data = self.make_fund_variables(data)
            kor_holidays = self.update_kor_holidays()

            ### 샌드위치 데이
            data = self.get_sandwich_score(data, 'is_holiday')

            # 샌드위치 - 첫날
            first = data['영업일자'].min()
            data.loc[data['영업일자'] == first, 'is_sandwich'] = self.get_sandwich_score_for_dates(first, kor_holidays)
            second = data['영업일자'].min() + timedelta(days = 1)
            data.loc[data['영업일자'] == second, 'is_sandwich'] = self.get_sandwich_score_for_dates(second, kor_holidays)

            # 샌드위치 - 마지막 날
            last = data['영업일자'].max()
            data.loc[data['영업일자'] == last, 'is_sandwich'] = self.get_sandwich_score_for_dates(last, kor_holidays)
            before = data['영업일자'].max() - timedelta(days = 1)
            data.loc[data['영업일자'] == before, 'is_sandwich'] = self.get_sandwich_score_for_dates(before, kor_holidays)

            # 샌드위치 포함한 공휴일
            data['is_holiday_sandwich'] = data['is_holiday'].astype(int) | (data['is_sandwich'] > 0).astype(int)

            ### 월별 가중치
            data = self.get_month_weights(data, monthly_weights)

            ### 요일별 가중치
            data = self.get_week_weights(data, weekly_weights)

            ### 직전 주차 평균
            sundays = data[data['weekday'] == "Sunday"][["영업일자", "영업장명_메뉴명"]].copy()
            for _, row in sundays.iterrows():
                date = row['영업일자']
                menu = row['영업장명_메뉴명']
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 7)
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 14)
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 21)

            ### 영업장명, 메뉴명 분리
            if '영업장명_메뉴명' in data.columns:
                data[['영업장명', '메뉴명']] = data['영업장명_메뉴명'].str.split('_', expand = True)

            ### 음수 처리
            negative = data[data['매출수량'] < 0]

            for idx, row in negative.iterrows():
                num = row['매출수량']
                if num < -10:
                    date = row['영업일자']
                    menu = row['영업장명_메뉴명']
                    prev_date = pd.to_datetime(date) - pd.Timedelta(days = 1)
                    prev_row = data[(data['영업일자'] == prev_date) & (data['영업장명_메뉴명'] == menu)]

                    if prev_row.iloc[0]["매출수량"] >= abs(num):
                        data.loc[prev_row.index[0], '매출수량'] += num

            # 남은 건 전부 0으로
            data.loc[data['매출수량'] < 0, '매출수량'] = 0

            return data

        # 예측하고자 하는 날들
        def make_variables_test(self, date, test_df, predict):
            """
            date : 최종 날짜 (입력 7일 중 가장 마지막) - TimeStamp
            test_df : 예측할 때 참고해올 데이터 -> 이거로 직전 주차 평균 생성
            """
            date = pd.to_datetime(date)
            future_dates = [date + timedelta(days = i + 1) for i in range(predict)]
            future_df = pd.DataFrame({'영업일자' : future_dates})

            menus_df = (test_df[['영업장명_메뉴명']].drop_duplicates().reset_index(drop = True))
            future_df = future_df.merge(menus_df, how='cross')

            kor_holidays = self.update_kor_holidays()

            # 기본적인 변수들
            future_df = self.make_fund_variables(future_df)

            future_df['영업일자'] = pd.to_datetime(future_df['영업일자']).dt.normalize()

            # 샌드위치
            future_df['is_sandwich'] = future_df['영업일자'].apply(lambda d: self.get_sandwich_score_for_dates(d, kor_holidays))

             # 샌드위치 포함한 공휴일
            future_df['is_holiday_sandwich'] = future_df['is_holiday'].astype(int) | (future_df['is_sandwich'] > 0).astype(int)

            # 월별 가중치
            future_df = self.get_month_weights(future_df, monthly_weights)

            # 요일별 가중치
            future_df = self.get_week_weights(future_df, weekly_weights)

            # 직전 주차 평균 -> 이거는 test 까지 받아오고 생각해야 함..
            sundays =  future_df.loc[future_df['weekday'] == "Sunday", ['영업일자', '영업장명_메뉴명']].copy()
            for _, row in sundays.iterrows():
                date = row['영업일자']
                menu = row['영업장명_메뉴명']
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 7)
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 14)
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 21)

            return future_df

In [ ]:
# 그냥 전부 만들면 돼
mv = Make_Variables()
data = mv.make_variables_train(data)

In [ ]:
print("기존 공휴일 : ", data['is_holiday'].sum().item())
print("샌드위치 : ", data['is_sandwich'].sum().item())
print("샌드위치 포함 공휴일 : ", data['is_holiday_sandwich'].sum().item())

기존 공휴일 :  33389
샌드위치 :  579
샌드위치 포함 공휴일 :  33968


In [ ]:
import pickle
data.to_pickle("/content/drive/MyDrive/3. Grad School/LG Aimers/DATA/train_data_new.pickle")

#### 저장된 데이터 불러오기

In [ ]:
# 데이터 불러오기
data = pd.read_pickle("/content/drive/MyDrive/3. Grad School/LG Aimers/DATA/train_data_new.pickle")

In [ ]:
def add_ohe_cols(data):
    season_order = ["Spring", "Summer", "Autumn", "Winter"]
    month_order = list(range(1,13))
    weekday_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

    data["season"] = pd.Categorical(data["season"], categories=season_order)
    data["month"]   = pd.Categorical(data["month"].astype(int), categories = month_order)
    data["weekday"] = pd.Categorical(data["weekday"], categories = weekday_order)

    season_ohe = pd.get_dummies(data["season"], prefix="season", dtype="int8")
    m_ohe  = pd.get_dummies(data["month"],   prefix="m",  dtype="int8")
    wd_ohe = pd.get_dummies(data["weekday"], prefix="wd", dtype="int8")

    return pd.concat([data, season_ohe, m_ohe, wd_ohe], axis=1)

In [ ]:
data_ohe = add_ohe_cols(data)
data_ohe.columns

Index(['영업일자', '영업장명_메뉴명', '매출수량', 'year', 'month', 'day', 'weekday',
       'weekday_enc', 'season', 'year_enc', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'is_holiday',
       'holiday_name', 'is_sandwich', 'is_holiday_sandwich', 'month_weight',
       'week_weight', 'prev_avg_7', 'prev_sd_7', 'prev_avg_14', 'prev_sd_14',
       'prev_avg_21', 'prev_sd_21', '영업장명', '메뉴명', 'season_Spring',
       'season_Summer', 'season_Autumn', 'season_Winter', 'm_1', 'm_2', 'm_3',
       'm_4', 'm_5', 'm_6', 'm_7', 'm_8', 'm_9', 'm_10', 'm_11', 'm_12',
       'wd_Monday', 'wd_Tuesday', 'wd_Wednesday', 'wd_Thursday', 'wd_Friday',
       'wd_Saturday', 'wd_Sunday'],
      dtype='object')

In [ ]:
import pickle
data_ohe.to_pickle("/content/drive/MyDrive/3. Grad School/LG Aimers/DATA/train_data_ohe.pickle")

In [42]:
cols =  ["year_enc", "month_sin", "month_cos", "day_sin", "day_cos", "weekday_sin", "weekday_cos", "is_holiday", "is_sandwich", "is_holiday_sandwich",
         "prev_avg_7", "prev_avg_14", "prev_avg_21", "prev_sd_7", "prev_sd_14", "prev_sd_21"]

###############
season_cols = [f"season_{s}" for s in ["Spring","Summer","Autumn","Winter"]]
m_cols  = [f"m_{i}" for i in range(1, 13)]
wd_cols = [f"wd_{d}" for d in ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]]
ohe_cols = season_cols + m_cols + wd_cols

cols = cols + ohe_cols
###############

# lstm에서는 minmaxscaling 해주는 게 안전
num_cols = ["prev_avg_7", "prev_avg_14", "prev_avg_21", "prev_sd_7", "prev_sd_14", "prev_sd_21"]

In [ ]:
data = data_ohe

#### 매출 여부 (분류)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
! pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.metrics import f1_score
from typing import Dict, List, Optional, Any
from collections import defaultdict
import pickle
import joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.3 MB/s eta 0:00:00


In [43]:
class ClassificationModel():
    def __init__(self, data = None, cols = None, enc_cols = None, model_path = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols or []

    def fit_model_cv(self, data : pd.DataFrame, cols : List[str], enc_cols : List[str] = None) -> pd.DataFrame:
        """
        data - 전체 dataset
        cols - 전체 변수들
        enc_cols - 인코딩 진행할 변수들 (범주형)
        """
        oof_parts = []
        enc_cols = enc_cols or []

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values('영업일자')

            # 나중에 oof 저장할 때 사용할 것들
            org_idx = group_df.index
            oof_proba = np.full(len(group_df), np.nan)

            # x, y 분리
            x = group_df[cols]
            y = group_df["매출_여부"]

            # time series split
            tscv = TimeSeriesSplit(n_splits = 2)

            # 각 split 별로
            for fold, (train_idx, val_idx) in enumerate(tscv.split(x)):

                x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
                x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

                # 수량 전부 0이거나 0 아닌 날 없으면 학습 불가
                if y_train.nunique() < 2:
                    print(f"{menu} 학습 불가")
                    continue

                if enc_cols:

                    # 범주형 변수 인코딩
                    target_encoder = TargetEncoder()
                    target_encoder.fit(x_train[enc_cols], y_train)

                    x_train = pd.concat([
                        target_encoder.transform(x_train[enc_cols]),
                        x_train[[c for c in cols if c not in enc_cols]]
                    ], axis = 1)

                    x_val = pd.concat([
                        target_encoder.transform(x_val[enc_cols]),
                        x_val[[c for c in cols if c not in enc_cols]]
                    ], axis = 1)

                # 모델 설정 - 불균형 대비
                pos = y_train.sum()
                neg = len(y_train) - pos
                spw = (neg / max(1, pos)) if pos > 0 else 1.0

                xgb_model = XGBClassifier(random_state = 1471, scale_pos_weight = spw, max_delta_step = 1, eval_metric = 'aucpr')

                # 모델 학습
                xgb_model.fit(x_train, y_train)
                oof_proba[val_idx] = xgb_model.predict_proba(x_val)[:, 1]

            part = pd.DataFrame({
                "index": org_idx.values,
                "영업장명_메뉴명": menu,
                "y_true": group_df["매출_여부"].values,
                "y_proba": oof_proba
            }).set_index("index")

            oof_parts.append(part)

        oof_df = pd.concat(oof_parts).sort_index()
        return oof_df


    def tuning_cv(self, data, cols, enc_cols, param_grid):
        """
        메뉴별로 하이퍼파라미터 튜닝 진행 (성능 보고.. 추가 진행? )
        """
        best_params_by_menu = {}

        return None


    def tune_threshold(self, oof_df : pd.DataFrame, metric = 'f1') -> Dict[str, float]:
        """
        메뉴별로 OOF 데이터프레임을 받아와서
        threshold를 최적화
        metric으로는 f1 사용
        """
        best_thresholds = {}
        all_thresholds = []

        for menu, group_df in oof_df.groupby("영업장명_메뉴명"):
            # CV 못한 메뉴들 따로
            mask = group_df["y_proba"].notna()
            if mask.sum() == 0:
                best_thresholds[menu] = None
                continue

            y_true = group_df.loc[mask, "y_true"].values
            y_proba = group_df.loc[mask, "y_proba"].values

            # 초기값 진행
            best_score, best_thr = -1, 0.5
            for thr in np.linspace(0.05, 0.95, 51):
                y_pred = (y_proba >= thr).astype(int)
                score = f1_score(y_true, y_pred, zero_division = 0)
                if score > best_score:
                    best_score = score
                    best_thr = thr

            best_thresholds[menu] = best_thr
            all_thresholds.append(best_thr)

        global_threshold = np.median(all_thresholds)

        for menu, threshold in best_thresholds.items():
            if threshold is None:
                best_thresholds[menu] = global_threshold

        return best_thresholds


    def get_final_model(self, data, cols, thresholds, enc_cols = None, hyperparameters_dict = None) -> Dict[str, Dict[str, Any]]:
        """
        메뉴별로 튜닝된 하이퍼파라미터, threshold를 반영하여
        최종 모델 적합
        """
        models = {}

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values('영업일자')

            # 범주형 변수 처리 - 전체 데이터로
            if enc_cols:
                target_encoder_full = TargetEncoder()
                group_df[enc_cols] = target_encoder_full.fit_transform(group_df[enc_cols], group_df['매출_여부'])

            # x, y 분리
            x_full = group_df[cols]
            y_full = group_df["매출_여부"]

            # 하이퍼파라미터
            hyperparameters = hyperparameters_dict.get(menu, {}) if hyperparameters_dict else {}

            # 모델 설정 - 불균형 대비
            pos = int(y_full.sum())
            neg = len(y_full) - pos
            spw = (neg / max(1, pos)) if pos > 0 else 1.0

            xgb_model_full = XGBClassifier(random_state = 1471, scale_pos_weight=spw, max_delta_step = 1, eval_metric = 'aucpr', **hyperparameters)

            # 모델 학습
            xgb_model_full.fit(x_full, y_full)

            models[menu] = {
                "model" : xgb_model_full,
                # "encoder" : target_encoder_full,
                "threshold" : thresholds.get(menu, 0.5)}

        return models

    def fit_whole_model(self, data, cols, enc_cols = None) -> Dict[str, Dict[str, Any]]:
        oof_df = self.fit_model_cv(data, cols, enc_cols)
        print("CV 완료!")
        thresholds = self.tune_threshold(oof_df)
        print("threshold 탐색 완료!")
        models = self.get_final_model(data, cols, thresholds, enc_cols)
        return models

    def save_cls_model(self, models, model_path):
        joblib.dump(models, model_path)
        print("모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

In [ ]:
classification = ClassificationModel()
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 8/cls_models.pkl'

data_zero = data.copy()
data_zero['매출_여부'] = data_zero['매출수량'].apply(lambda x:1 if x > 0 else 0)

models = classification.fit_whole_model(data = data_zero, cols = cols, enc_cols = None)
classification.save_cls_model(models, model_path)

담하_(단체) 생목살 김치전골 2.0 학습 불가
담하_갱시기 학습 불가
담하_꼬막 비빔밥 학습 불가
담하_더덕 한우 지짐 학습 불가
담하_명인안동소주 학습 불가
담하_문막 복분자 칵테일 학습 불가
라그로타_AUS (200g) 학습 불가
라그로타_그릴드 비프 샐러드 학습 불가
라그로타_까르보나라 학습 불가
라그로타_모둠 해산물 플래터 학습 불가
라그로타_버섯 크림 리조또 학습 불가
라그로타_시저 샐러드  학습 불가
라그로타_알리오 에 올리오  학습 불가
라그로타_양갈비 (4ps) 학습 불가
라그로타_한우 (200g) 학습 불가
라그로타_해산물 토마토 스튜 파스타 학습 불가
미라시아_(오븐) 하와이안 쉬림프 피자 학습 불가
미라시아_잭 애플 토닉 학습 불가
미라시아_콥 샐러드 학습 불가
연회장_공깃밥 학습 불가
연회장_마라샹궈 학습 불가
연회장_삼겹살추가 (200g) 학습 불가
연회장_왕갈비치킨 학습 불가
카페테리아_진사골 설렁탕 학습 불가
CV 완료!
threshold 탐색 완료!
모델 저장 완료!


In [44]:
# 다시 불러오기
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 8/cls_models.pkl'
classification = ClassificationModel()
models_class = classification.load_saved_model(model_path)

#### 매출 예측 (회귀)

In [ ]:
from xgboost import XGBRegressor
# ! pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.model_selection import TimeSeriesSplit
from collections import defaultdict
from itertools import product
import pickle
import joblib

In [45]:
class RegressionModel():
    def __init__(self, data = None, cols = None, enc_cols = None, model_path = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols or []

    def smape_score(self, y_true, y_pred, eps = 1e-8):
        y_true = np.asarray(y_true, dtype=float)
        y_pred = np.asarray(y_pred, dtype=float)
        denom = np.abs(y_true) + np.abs(y_pred)
        denom = np.where(denom < eps, eps, denom)
        return 200.0 * np.mean(np.abs(y_true - y_pred) / denom)

    def tuning_cv(self, data : pd.DataFrame, cols : List[str], enc_cols : List[str] = None, param_grid = None) -> Dict[str, Dict]:
        """
        메뉴별로 하이퍼파라미터 튜닝 진행
        """
        best_params_by_menu = {}

        if param_grid is None:
            param_grid = {
                "min_child_weight" : [1, 5],
                "max_depth" : [5, 6, 7],
                "subsample" : [0.7, 0.9],
                "n_estimators" : [100, 150],
                "learning_rate" : [0.03, 0.05]}

        keys = list(param_grid.keys())
        combos = list(product(*[param_grid[k] for k in keys]))

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values("영업일자")

            x = group_df[cols]
            y = group_df["매출수량"]

            # time series split
            tscv = TimeSeriesSplit(n_splits = 3)

            # 초기값
            best_smape = np.inf
            best_params = None

            for values in combos:
                params = dict(zip(keys, values))

                base_params = {"random_state" : 1478}
                base_params.update(params)

                fold_smapes : List[float] = []

                for fold, (train_idx, val_idx) in enumerate(tscv.split(x)):

                    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
                    x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

                    # 데이터 수 적으면 학습 불가
                    if len(x_train) < 20:
                        print(f"{menu} 튜닝 불가)")
                        continue

                    # 범주형 변수 인코딩
                    if enc_cols:
                        target_encoder = TargetEncoder()
                        target_encoder.fit(x_train[enc_cols], y_train)

                        x_train = pd.concat([
                            target_encoder.transform(x_train[enc_cols]),
                            x_train[[c for c in cols if c not in enc_cols]]
                        ], axis = 1)

                        x_val = pd.concat([
                            target_encoder.transform(x_val[enc_cols]),
                            x_val[[c for c in cols if c not in enc_cols]]
                        ], axis = 1)

                    # 모델 설정
                    xgb_model = XGBRegressor(**base_params)

                    # 모델 학습
                    xgb_model.fit(x_train, y_train)

                    pred = xgb_model.predict(x_val)
                    smape = self.smape_score(y_val, pred)
                    fold_smapes.append(smape)

                mean_smape = float(np.mean(fold_smapes))

                if mean_smape < best_smape:
                    best_smape = mean_smape
                    best_params = base_params

            if best_params is not None:
                best_params_by_menu[menu] = best_params

        return best_params_by_menu


    def get_final_model(self, data, cols, enc_cols, hyperparameters_dict : dict[str, dict[str, Any]] = None)  -> Dict[str, Dict[str, Any]]:
        """
        data, validation_reg - train, validation dataset
        cols - 전체 변수들
        enc_cols - 인코딩 진행할 변수들 (범주형)
        """
        models = {}

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values("영업일자")

            # 데이터 수 적으면 학습 불가
            if len(group_df) < 10:
                print(f"{menu} 학습 불가")
                continue

            # 범주형 변수 처리
            if enc_cols:
                target_encoder_full = TargetEncoder()
                group_df[enc_cols] = target_encoder_full.fit_transform(group_df[enc_cols], group_df['매출수량'])

            # x, y 분리
            x_full = group_df[cols]
            y_full = group_df["매출수량"]

            # 하이퍼파라미터 불러오기
            params = hyperparameters_dict.get(menu, {}) if hyperparameters_dict else {}

            base_params = {"random_state" : 1478}
            base_params.update(params)

            # 모델 설정
            xgb_model_full = XGBRegressor(**base_params)

            # 모델 학습
            xgb_model_full.fit(x_full, y_full)

            models[menu] = {
                "model" : xgb_model_full,
                "hyperparameters" : base_params,
                # "encoder" : target_encoder_full
            }

        return models

    def fit_whole_model(self, data, cols, enc_cols = None) -> Dict[str, Dict[str, Any]]:
        best_params_by_menu = self.tuning_cv(data, cols, enc_cols)
        print("튜닝 완료!")
        models = self.get_final_model(data, cols, enc_cols, best_params_by_menu)
        return models

    def save_reg_model(self, models, model_path):
        joblib.dump(models, model_path)
        print("모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

In [ ]:
regression = RegressionModel()
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 8/reg_models.pkl'

data_notzero = data[data['매출수량'] > 0]

models = regression.fit_whole_model(data = data_notzero, cols = cols, enc_cols = None)
regression.save_reg_model(models, model_path)

튜닝 완료!
모델 저장 완료!


In [46]:
# 다시 불러오기
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 8/reg_models.pkl'
regression = RegressionModel()
models_reg = regression.load_saved_model(model_path)

#### 매출 예측 (시계열)

In [47]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import random
import glob
import joblib

import torch
import torch.nn as nn
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda": torch.cuda.empty_cache()

In [48]:
### Random Seed & Parameters
def set_seed(seed = 1471):
    random.seed(seed) # 일반 seed
    np.random.seed(seed) # numpy 난수 고정
    torch.manual_seed(seed) # CPU 난수 고정
    os.environ["PYTHONHASHSEED"] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(1478)

##### Loss Functions

In [49]:
# weight 직접 입력해서 반영 가능
class WeightedSMAPELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps

    def forward(self, y_pred, y_true, w=None):
        num = (y_pred - y_true).abs()
        den = (y_pred.abs() + y_true.abs()).clamp(min=self.eps)
        smape = 2.0 * num / (den)

        mask = (y_true != 0).float()
        if w is None:
            w = torch.ones_like(y_true)
        else:
            if w.dim() == 1:
                w = w.view(-1, 1)
            w = w.expand_as(y_true)

        w_mask = w * mask
        denom = w_mask.sum().clamp(min=self.eps)
        loss = (smape * w_mask).sum() / denom
        return loss

# weighted huberLoss
class WeightedHuberLoss(nn.Module):
    def __init__(self, delta = 1.0, zeros : bool = False, eps = 1e-8):
        """
        delta -L2 -> L1로 전환되는 임계값
        zeros - y_true = 0인 샘플 제외할지 여부
        eps - 분모가 0이 되는 것을 방지하기 위한 작은 값
        """
        super().__init__()
        self.delta = float(delta)
        self.zeros = bool(zeros)
        self.eps = float(eps)

    def forward(self, y_pred, y_true, w = None):
        """
        w -  (N, ) 형태로 된 가중치
        """
        error = y_pred - y_true
        abs_error = error.abs()
        huber = torch.where(
            abs_error <= self.delta,
            0.5 * error**2,
            self.delta * (abs_error - 0.5 * self.delta),
        )

        if self.zeros:
            mask = (y_true != 0).float()
        else:
            mask = torch.ones_like(y_true)

        if w is None:
            w_full = torch.ones_like(y_true)
        else:
            if w.dim() == 1:
                w = w.view(-1, 1) # (N, 1)로 확장
            w_full = w.expand_as(y_true).float()

        w_mask = w_full * mask
        denom = w_mask.sum().clamp(min=self.eps)
        return (huber * w_mask).sum() / denom

# SMAPE + Huber Loss
class CombinationLoss(nn.Module):
    def __init__(self, losses, weights):
        """
        losses - 결합할 손실 리스트
        weights - 각 손실의 가중치 리스트
        """
        super().__init__()
        if not losses or len(losses) != len(weights):
            raise ValueError("loss 배열과 weight 배열의 길이가 다릅니다.")
        self.losses = nn.ModuleList(losses)

        weight_tensor = torch.tensor(weights, dtype=torch.float32)
        self.weights = weight_tensor / weight_tensor.sum()

    def forward(self, y_pred, y_true, w = None):
        """
        w - 공통 가중치
        """
        total_loss = 0.0
        device_weights = self.weights.to(y_pred.device)
        for i, lf in enumerate(self.losses):
            loss = lf(y_pred, y_true, w)
            total_loss += device_weights[i] * loss
        return total_loss

In [ ]:
def build_menu_weights(
    df: pd.DataFrame,
    menu_key: str = "영업장명_메뉴명",
    target_col: str = "매출수량",
    dispersion: str = "p95-p5",   # "range" | "iqr" | "std" | "p95-p5"
    alpha: float = 1.0,            # 민감도 (↑ 크면 가중치 차이 커짐)
    clip: tuple = (0.25, 4.0),     # 과도한 비율 방지
    eps: float = 1e-6
) -> dict:
    """
    메뉴별 난이도(분산/범위) 기반 가중치 (간극 ↑ -> 가중치 ↑)
    반환: {menu_key -> weight}
    """
    disp_vals = {}
    for k, g in df.groupby(menu_key):
        y = g[target_col].to_numpy()
        if len(y) == 0:
            disp = 0.0
        else:
            if dispersion == "range":
                disp = float(np.max(y) - np.min(y))
            elif dispersion == "iqr":
                q75, q25 = np.percentile(y, [75, 25])
                disp = float(q75 - q25)
            elif dispersion == "std":
                disp = float(np.std(y))
            else:  # "p95-p5"
                q95, q5 = np.percentile(y, [95, 5])
                disp = float(q95 - q5)
        disp_vals[k] = disp

    # 분포 → 가중치 (단조증가), 평균 1로 정규화
    disp_series = pd.Series(disp_vals)
    # 0도 있을 수 있으니 +eps, 로그스케일 쓰고 싶으면 바꿔도 됨
    raw_w = (disp_series + eps) ** alpha
    w = raw_w / raw_w.mean()

    # 과도한 값 클램프 (옵션)
    w = w.clip(lower=clip[0], upper=clip[1])
    return w.to_dict()

def build_store_weights(
    df: pd.DataFrame,
    store_col: str = "영업장명",
    menu_col: str = "메뉴명",      # 없으면 아래에서 자동 fallback
    combined_key: str = "영업장명_메뉴명",
    beta: float = 1.0,             # 메뉴 가짓수 민감도
    clip: tuple = (0.5, 3.0)
) -> dict:
    """
    음식점별 메뉴 가짓수 기반 가중치 (가짓수 ↑ -> 가중치 ↑)
    반환: {store -> weight}
    """
    if (store_col in df.columns) and (menu_col in df.columns):
        n_menu = df.groupby(store_col)[menu_col].nunique()
    else:
        # fallback: combined_key에서 store를 추출 (최초 '_' 기준 split)
        def get_store(x: str) -> str:
            # 필요하면 사용자 규칙에 맞게 수정
            return str(x).split("_", 1)[0]
        tmp = df[[combined_key]].copy()
        tmp[store_col] = tmp[combined_key].astype(str).map(get_store)
        # store별 unique combined_key 개수 = 메뉴 가짓수로 근사
        n_menu = tmp.groupby(store_col)[combined_key].nunique()

    raw_w = (n_menu.astype(float) + 1e-6) ** beta
    w = raw_w / raw_w.mean()
    w = w.clip(lower=clip[0], upper=clip[1])
    return w.to_dict()

##### LSTMModel

In [50]:
class MultiOutputLSTM(nn.Module):
        def __init__(self, input_dim = 1, hidden_dim = 256, num_layers = 4, output_dim = 7, dropout = 0.4):
            """ 7개 값 예측 (PREDICT 만큼의 날짜의 값을 예측하고자 함)"""
            super(MultiOutputLSTM, self).__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first = True, dropout = dropout)
            self.fc = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            out, _ = self.lstm(x)
            return self.fc(out[:, -1, :]) # 마지막 시점 출력만 선택해서 fc에 넣음 -> (batch * output_dim)

In [51]:
class LSTMModel():
    def __init__(self, data = None, cols = None, enc_cols = None, num_cols = None, scaler = None,
                 lookback = 28, predict = 7, hidden_dim = 256, num_layers = 4, device = "cuda", epochs = 200, batch_size = 16, dropout = 0.4,
                 delta = 0.1, val_ratio = 0.2, horizon_weight_mode = "linear",
                 lr = 0.04, losses = None, loss_weights = [0.5, 0.5], menu_weights = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols
        self.scaler = scaler

        self.lookback = lookback
        self.predict = predict
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.device = torch.device(device)
        self.epochs = epochs
        self.batch_size = batch_size
        self.dropout = dropout

        self.delta = delta
        self.val_ratio = val_ratio
        self.horizon_weight_mode = horizon_weight_mode
        self.lr = float(lr)
        self.losses = losses
        self.loss_weights = loss_weights
        self.menu_weights = menu_weights

    # enc_cols는 LabelEncoding
    def label_encoding_lstm(self, data, enc_cols = None):
        encoders = {}
        if enc_cols:
            for col in enc_cols:
                if data[col].dtype == 'object' or data[col].dtype.name == 'bool' or data[col].dtype.name == 'category':
                    le = LabelEncoder()
                    data[col] = le.fit_transform(data[col])
                    encoders[col] = le
            return data, encoders

    def minmax_scaling_features(self, data, num_cols):
        scaler = MinMaxScaler()
        data[num_cols] = scaler.fit_transform(data[num_cols])
        return data, scaler

    # 매출수량은 MinMaxScaling
    def minmax_scaling_target(self, data):
        scaler = MinMaxScaler()
        data['매출수량'] = scaler.fit_transform(data[['매출수량']])
        return data, scaler

    def build_horizon_weights(self, predict = 7):
        """
        수평선(예측일) 가중치 벡터 생성.
        - 'linear': 1..predict 선형 가중 후 정규화
        - 'uniform': 동일 가중
        """
        if self.horizon_weight_mode == "uniform":
            w = torch.ones(predict)
        else:
            # 기본: 뒤쪽일수록 더 큰 가중
            w = torch.arange(1, predict + 1).float()
        w = w / w.sum()
        return w

    def compute_smape(self, y_pred, y_true, eps=1e-8):
        """정규화 스케일에서의 SMAPE( y_true==0 은 마스킹 )"""
        num = (y_pred - y_true).abs()
        den = (y_pred.abs() + y_true.abs()).clamp(min=eps)
        smape = 2.0 * num / den
        mask = (y_true != 0).float()
        denom = mask.sum().clamp(min=eps)
        return (smape * mask).sum().item() / denom.item()

    def _tss_last_split_indices(self, n_samples: int, n_splits: int = 3):
        """
        마지막 TimeSeriesSplit fold의 (train_idx, val_idx)만 반환.
        누수 방지를 위해 gap = lookback + predict - 1 권장.
        """
        idx = np.arange(n_samples)
        tscv = TimeSeriesSplit(n_splits=n_splits, gap=self.lookback + self.predict - 1)
        tr_idx, va_idx = None, None
        for tr, va in tscv.split(idx):
            tr_idx, va_idx = tr, va
        return tr_idx, va_idx

    def _make_windows(self, x: np.ndarray, y: np.ndarray):
        """
        구간 내부에서만 윈도우 생성 (누수 없음)
        x: (T, F), y: (T,)
        return: x:(N, lookback, F), y:(N, predict) or (None, None)
        """
        T = len(x)
        xs, ys = [], []
        for i in range(T - self.lookback - self.predict + 1):
            xs.append(x[i:i+self.lookback])
            ys.append(y[i+self.lookback:i+self.lookback+self.predict])
        if not xs:
            return None, None
        return np.stack(xs).astype(np.float32), np.stack(ys).astype(np.float32)

#############################################################################
    def train_lstm(self, train_df, cols, enc_cols, num_cols,
                   device, epochs, batch_size, lr, dropout,
                   losses, loss_weights,
                   n_splits : int = 3, print_every = 50):
        """
        영업장, 메뉴별로 LSTM 모델 훈련, 각각을 trained_models에 저장
        Loss - CombinationLoss([WeightedSMAPELoss(), WeightedHuberLoss()], [0.5, 0.5]
        """
        trained_models = {}
        horizon_w = self.build_horizon_weights(self.predict).to(device)

        # Loss 구성
        comb_loss = CombinationLoss(losses = losses, weights = loss_weights).to(device)

        # store_menu : 영업장명_메뉴명 / group : 나머지 데이터
        for store_menu, group in tqdm(train_df.groupby(["영업장명_메뉴명"]), desc = "Training LSTM"):

            # 날짜 순으로 정렬해서 데이터가 너무 적으면 -> 학습 생략
            store_train = group.sort_values("영업일자").copy()
            if len(store_train) < self.lookback + self.predict:
                continue

            # ===== TimeSeriesSplit: 마지막 폴드를 검증으로 사용 =====
            tr_idx, va_idx = self._tss_last_split_indices(len(store_train), n_splits=n_splits)
            df_tr = store_train.iloc[tr_idx].copy()
            df_va = store_train.iloc[va_idx].copy()

            # ===== 인코더/스케일러는 train(df_tr)에만 fit, df_tr/df_va에 transform (누수 방지) =====
            encoders = {}
            if enc_cols:
                for c in enc_cols:
                    le = LabelEncoder()
                    df_tr[c] = le.fit_transform(df_tr[c].astype(str))
                    # val에서 미지 카테고리 안전 처리
                    cls2idx = {cls: i for i, cls in enumerate(le.classes_)}
                    df_va[c] = df_va[c].astype(str).map(lambda v: cls2idx.get(v, -1))
                    encoders[c] = le

            features_scaler = None
            if num_cols:
                features_scaler = MinMaxScaler()
                df_tr[num_cols] = features_scaler.fit_transform(df_tr[num_cols])
                df_va[num_cols] = features_scaler.transform(df_va[num_cols])

            target_scaler = MinMaxScaler()
            df_tr[['매출수량']] = target_scaler.fit_transform(df_tr[['매출수량']])
            df_va[['매출수량']] = target_scaler.transform(df_va[['매출수량']])

            # ===== 원시 배열 =====
            X_tr = df_tr[cols].to_numpy(dtype=np.float32)
            y_tr = df_tr['매출수량'].to_numpy(dtype=np.float32)
            X_va = df_va[cols].to_numpy(dtype=np.float32)
            y_va = df_va['매출수량'].to_numpy(dtype=np.float32)

            # ===== 세그먼트 내부에서만 윈도우 생성 (경계 누수 없음) =====
            x_tr_np, y_tr_np = self._make_windows(X_tr, y_tr)
            x_va_np, y_va_np = self._make_windows(X_va, y_va)
            if x_tr_np is None or x_va_np is None:
                continue

            # 텐서 이동
            x_train = torch.tensor(x_tr_np).to(device)         # (N_tr, lookback, F)
            y_train = torch.tensor(y_tr_np).to(device)         # (N_tr, predict)
            x_val   = torch.tensor(x_va_np).to(device)
            y_val   = torch.tensor(y_va_np).to(device)

            # 모델 초기화 (영업장_메뉴별로 다른 모델)
            model = MultiOutputLSTM(input_dim = len(cols),
                                    hidden_dim = self.hidden_dim,
                                    num_layers = self.num_layers,
                                    output_dim = self.predict,
                                    dropout = dropout).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr = lr)
            best_val = float("inf")
            best_sd = None
            best_epoch = -1

            # 학습 모드로 설정
            model.train()

            # epochs 만큼 훈련
            for epoch in range(1, epochs + 1):
                model.train(True)
                perm = torch.randperm(len(x_train), device = device)
                epoch_train_loss = 0.0
                total_train_count = 0

                # idx : 랜덤하게 섞인 index들
                for i in range(0, len(x_train), batch_size):
                    batch_idx = perm[i : i+batch_size] # 배치 개수만큼 끊어서
                    x_batch, y_batch = x_train[batch_idx], y_train[batch_idx] # 배치 데이터 할당

                    weight_mat = horizon_w.unsqueeze(0).repeat(y_batch.size(0), 1)

                    output = model(x_batch) # 모델 태워서
                    loss = comb_loss(output, y_batch, w = weight_mat) # 평가하고

                    optimizer.zero_grad() # 역전파를 위한 초기화
                    loss.backward() # 역전파
                    optimizer.step() # 최적화

                    epoch_train_loss += loss.item() * y_batch.size(0)
                    total_train_count += y_batch.size(0)

                epoch_train_loss /= max(1, total_train_count)

                ### Validation
                with torch.no_grad():
                    model.eval()
                    pred_val = model(x_val)
                    weight_val = horizon_w.unsqueeze(0).repeat(y_val.size(0), 1)
                    val_loss = comb_loss(pred_val, y_val, w = weight_val).item()
                    val_smape = self.compute_smape(pred_val, y_val)

                    if epoch % 50 == 0:
                        print(f"\n[{store_menu}] Epoch {epoch} ==============================\n "
                            f"Train Loss {epoch_train_loss:.5f} | Val Loss {val_loss:.5f} | SMAPE {val_smape:.5f}")

                    if val_loss < best_val - 1e-6:
                        best_val = val_loss
                        best_sd = {k : v.detach().cpu().clone() for k, v in model.state_dict().items()}
                        best_epoch = epoch

                model.train(True)


            if best_sd is not None:
                model.load_state_dict(best_sd)

            # 모델 저장
            trained_models[store_menu] = {
                'model': model.state_dict(),
                # 'encoders' : encoders,
                'features_scaler' : features_scaler,
                'target_scaler': target_scaler,
                'meta' : {
                    'input_dim': len(cols),
                    'hidden_dim': self.hidden_dim,
                    'num_layers': self.num_layers,
                    'output_dim': self.predict,
                    'dropout' : self.dropout
                }
                }

        return trained_models

    def save_lstm_model_gpu(self, models, model_path):
        joblib.dump(models, model_path)
        print("GPU 버전 모델 저장 완료!")

    def save_lstm_model_cpu(self, models, model_path):
        cpu_models = {}
        for k, bundle in models.items():
            cpu_models[k] = {
            'model': {kk: vv.cpu() for kk, vv in bundle['model'].items()},  # 모델만 CPU로
            # 'encoders': bundle['encoders'],
            'features_scaler': bundle['features_scaler'],
            'target_scaler': bundle['target_scaler'],
            'meta' : bundle['meta']
        }
        joblib.dump(cpu_models, model_path)
        print("CPU 버전 모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

##### Train

In [ ]:
# 데이터 준비
data["is_sandwich_norm"] = data["is_sandwich"].fillna(0) / 5.0

features = cols + ["매출수량", "영업일자", "영업장명_메뉴명", "is_sandwich_norm"]
dataset_lstm = data[features]

losses = [WeightedSMAPELoss(eps = 1e-8), WeightedHuberLoss(delta = 1.0, zeros = False, eps = 1e-8)]
loss_weights = [0.5, 0.5]
lstm = LSTMModel(lookback = 28, hidden_dim = 128, num_layers = 3, predict = 7, dropout = 0.3)

lookback, predict, batch_size, epochs = 28, 7, 16, 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_lstm = lstm.train_lstm(train_df = dataset_lstm, cols = cols, enc_cols = None, num_cols = num_cols,
                               device = device, epochs = epochs, batch_size = batch_size, lr = 1e-3, dropout = 0.4,
                               losses = losses, loss_weights = loss_weights, n_splits = 3, print_every = 50)

Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]


[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 50 ==============================
 Train Loss 0.20157 | Val Loss 0.36097 | SMAPE 0.70360

[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 100 ==============================
 Train Loss 0.15947 | Val Loss 0.36834 | SMAPE 0.72988

[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 150 ==============================
 Train Loss 0.10027 | Val Loss 0.34908 | SMAPE 0.67813


Training LSTM:   1%|          | 1/193 [00:21<1:09:18, 21.66s/it]


[('느티나무 셀프BBQ_1인 수저세트',)] Epoch 200 ==============================
 Train Loss 0.07661 | Val Loss 0.35232 | SMAPE 0.69164

[('느티나무 셀프BBQ_BBQ55(단체)',)] Epoch 50 ==============================
 Train Loss 0.16365 | Val Loss 0.29542 | SMAPE 0.57905

[('느티나무 셀프BBQ_BBQ55(단체)',)] Epoch 100 ==============================
 Train Loss 0.10423 | Val Loss 0.29348 | SMAPE 0.57285

[('느티나무 셀프BBQ_BBQ55(단체)',)] Epoch 150 ==============================
 Train Loss 0.07261 | Val Loss 0.30733 | SMAPE 0.59615


Training LSTM:   1%|          | 2/193 [00:48<1:18:35, 24.69s/it]

...


[('화담숲카페_아메리카노 ICE',)] Epoch 200 ==============================
 Train Loss 0.16175 | Val Loss 0.36602 | SMAPE 0.71656

[('화담숲카페_카페라떼 ICE',)] Epoch 50 ==============================
 Train Loss 0.21754 | Val Loss 0.30307 | SMAPE 0.57300

[('화담숲카페_카페라떼 ICE',)] Epoch 100 ==============================
 Train Loss 0.16350 | Val Loss 0.33444 | SMAPE 0.61312

[('화담숲카페_카페라떼 ICE',)] Epoch 150 ==============================
 Train Loss 0.12993 | Val Loss 0.33526 | SMAPE 0.63653


Training LSTM:  99%|█████████▉| 192/193 [1:06:36<00:20, 20.70s/it]


[('화담숲카페_카페라떼 ICE',)] Epoch 200 ==============================
 Train Loss 0.10311 | Val Loss 0.32563 | SMAPE 0.61329

[('화담숲카페_현미뻥스크림',)] Epoch 50 ==============================
 Train Loss 0.22601 | Val Loss 0.39204 | SMAPE 0.75036

[('화담숲카페_현미뻥스크림',)] Epoch 100 ==============================
 Train Loss 0.17498 | Val Loss 0.39714 | SMAPE 0.77516

[('화담숲카페_현미뻥스크림',)] Epoch 150 ==============================
 Train Loss 0.15189 | Val Loss 0.42273 | SMAPE 0.81154


Training LSTM: 100%|██████████| 193/193 [1:06:56<00:00, 20.81s/it]


[('화담숲카페_현미뻥스크림',)] Epoch 200 ==============================
 Train Loss 0.11965 | Val Loss 0.40433 | SMAPE 0.77351


In [ ]:
hidden_dim = 128
num_layers = 3
model_path = f'/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 8/lstm_models_{hidden_dim}_{num_layers}.pkl'
lstm.save_lstm_model_cpu(trained_lstm, model_path)

CPU 버전 모델 저장 완료!


#### 예측하기

In [53]:
# 저장된 모델 로드 - CPU
lookback, predict, batch_size, epochs = 28, 7, 16, 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 8/lstm_models_128_3.pkl'
lstm = LSTMModel(lookback = 28, predict = 7)
trained_lstm = lstm.load_saved_model(model_path)

In [54]:
class PredictionFunctions():
    def __init__(self, test_df = None, trained_models = None, test_prefix = None, cols = None, enc_cols = None, lookback = 28, predict = 7):
        self.test_df = test_df
        self.trained_models = trained_models
        self.test_prefix = test_prefix
        self.cols = cols
        self.enc_cols = enc_cols or []
        self.lookback = lookback
        self.predict = predict

    def predict_class(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출여부] DataFrame
        """
        results = []

        for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
            store_menu = store_menu_tup[0]
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델 불러오기
            model = trained_models[store_menu]["model"]
            encoder = trained_models[store_menu].get('encoder', None)
            threshold = trained_models[store_menu].get("threshold", 0.5)

            # 변수 추가하기
            mv = Make_Variables()
            store_test['영업일자'] = pd.to_datetime(store_test['영업일자'])
            store_test_sorted = store_test.sort_values('영업일자')
            last_date = store_test_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_test, predict = 7)
            if enc_cols:
                encoded = encoder.transform(future_df[enc_cols])
                encoded_df = pd.DataFrame(encoded, columns = enc_cols, index = future_df.index)
                future_df[enc_cols] = encoded_df
            x = future_df[cols]

            if hasattr(model, "classes_"):
                pos_idx = int(np.where(model.classes_ == 1)[0][0])
            else:
                pos_idx = 1

            proba = model.predict_proba(x)[:, pos_idx]
            y_hat = (proba >= threshold).astype(int)


            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, y_hat):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출여부': val
                })

        return pd.DataFrame(results)

    def predict_reg(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """
        results = []

        for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
            store_menu = store_menu_tup[0]
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델 불러오기
            model = trained_models[store_menu]["model"]
            encoder = trained_models[store_menu].get('encoder', None)

            # 변수 추가하기
            mv = Make_Variables()
            store_test['영업일자'] = pd.to_datetime(store_test['영업일자'])
            store_test_sorted = store_test.sort_values('영업일자')
            last_date = store_test_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_test, predict = 7)
            if enc_cols:
                encoded = encoder.transform(future_df[enc_cols])
                encoded_df = pd.DataFrame(encoded, columns = enc_cols, index = future_df.index)
                future_df[enc_cols] = encoded_df
            future_df = future_df[cols]

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, model.predict(future_df)):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량': val
                })

        return pd.DataFrame(results)

    def predict_lstm(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, num_cols : list, lookback = 28, predict = 7, device = None):
        """
        Input : test_df - test data, trained_models - list(menu : { model}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """

        if device is None:
            device = getattr(self, "device", "cpu")
        device = torch.device(device)

        results = []

        # 매장, 메뉴별로 그룹화해서 예측
        for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델, scaler 불러오기
            state = trained_models[store_menu]['model']
            meta = trained_models[store_menu]['meta']
            encoders = trained_models[store_menu].get('encoders', None)
            features_scaler = trained_models[store_menu]['features_scaler']
            target_scaler = trained_models[store_menu]['target_scaler']

            # LSTM 모델 생성
            model = MultiOutputLSTM(
                input_dim=meta['input_dim'],
                hidden_dim=meta['hidden_dim'],
                num_layers=meta['num_layers'],
                output_dim=meta['output_dim']
            )

            model.load_state_dict(state)
            model = model.to(device).eval()

            # LSTM 입력으로 활용할 최근 lookback 만큼의 데이터 가져오기
            mv = Make_Variables()
            store_test = store_test.sort_values('영업일자')
            store_test = mv.make_variables_train(data = store_test)
            store_test_sorted = store_test.sort_values('영업일자')

            features = cols + ["매출수량"]
            if len(store_test_sorted) < lookback:
                continue

            recent_df = store_test_sorted[features].iloc[-lookback:].copy()
            if len(recent_df) < lookback:
                continue # lookback 만큼의 데이터가 없으면 예측 안 하고 넘어가기

            ##### 요기서 변수 추가
            last_date = store_test_sorted['영업일자'].iloc[-1]
            recent_df_for_mv = store_test_sorted[features + ['영업장명_메뉴명', '영업일자']].iloc[-lookback:].copy()
            future_df = mv.make_variables_test(date = last_date, test_df = recent_df_for_mv, predict = 7)
            future_df['매출수량'] = 0.0
            full_df = pd.concat([recent_df, future_df[features]], axis = 0)

            # enc_cols 정규화
            if enc_cols:
                for col in enc_cols:
                    if col not in full_df.columns:
                        continue
                    if col in encoders:
                        le = encoders[col]
                        full_df[col] = le.transform(full_df[col].astype(str)) ##### str로 바꿔봄...
                    else:
                        full_df[col] = full_df[col].astype(int)

            # num_cols 스케일링
            full_df[num_cols] = features_scaler.transform(full_df[num_cols])

            x_input_vals = full_df[cols].values
            x_input = x_input_vals[:lookback]
            x_input = torch.tensor([x_input]).float().to(device)

            # 예측 수행
            with torch.no_grad():
                pred_scaled = model(x_input).squeeze().cpu().numpy()

            # 역정규화
            restored = []
            for i in range(predict):
                dummy = np.zeros((1, len(features)))
                dummy[0, features.index("매출수량")] = pred_scaled[i]
                restored_val = target_scaler.inverse_transform(dummy)[0, features.index("매출수량")]
                restored.append(max(restored_val, 0)) # 음수 나오면 0으로 처리

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, restored):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량(lstm)': val
                })

        return pd.DataFrame(results)

#### 예측값 생성

In [ ]:
import re
all_preds_class = []
all_preds_reg = []
all_preds_lstm = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))
predictions = PredictionFunctions()

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # lstm 넣고
    pred_lstm = predictions.predict_lstm(test_df, trained_lstm, test_prefix, cols, enc_cols = None, num_cols, device = 'cpu')
    all_preds_lstm.append(pred_lstm)

df_lstm  = pd.concat(all_preds_lstm, ignore_index=True)

In [ ]:
len(df_lstm)

13510

In [55]:
import re
all_preds_class = []
all_preds_reg = []
all_preds_lstm = []


# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))
predictions = PredictionFunctions()

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # 일단 분류 모델 넣고
    pred_class = predictions.predict_class(test_df, models_class, test_prefix, cols, enc_cols = None)
    all_preds_class.append(pred_class)

    # 1 나오면 회귀 모델 넣고
    pred_reg = predictions.predict_reg(test_df, models_reg, test_prefix, cols, enc_cols = None)
    all_preds_reg.append(pred_reg)

    # lstm 넣고
    pred_lstm = predictions.predict_lstm(test_df, trained_lstm, test_prefix, cols, enc_cols = None, num_cols = num_cols)
    all_preds_lstm.append(pred_lstm)

    print("예측 완료 !")

df_class = pd.concat(all_preds_class, ignore_index = True)
df_reg   = pd.concat(all_preds_reg, ignore_index = True)
df_lstm  = pd.concat(all_preds_lstm, ignore_index = True)

/tmp/ipython-input-4146410540.py:178: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_input = torch.tensor([x_input]).float().to(device)


예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !
예측 완료 !


In [66]:
full_pred_df = pd.merge(df_class, df_reg, on=['영업일자', '영업장명_메뉴명'], how='outer')
full_pred_df.rename(columns={'매출수량': '매출수량(reg)'}, inplace=True)

df_lstm_plz = df_lstm.copy()
df_lstm_plz['영업장명_메뉴명'] = df_lstm['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

full_pred_df = pd.merge(df_lstm_plz, full_pred_df, on=['영업일자', '영업장명_메뉴명'], how='outer')
full_pred_df.head()

,영업일자,영업장명_메뉴명,매출수량(lstm),매출여부,매출수량(reg)
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,10.007466,1,12.256191
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),32.985085,0,78.131004
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",7.921399,1,9.553818
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",4.646937,1,1.381822
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",1.028644,1,1.663972


In [67]:
full_pred_df['매출수량'] = np.where(
    full_pred_df['매출여부'] == 1,
    full_pred_df['매출수량(reg)'] * 0.35 + full_pred_df['매출수량(lstm)'] * 0.65,
    full_pred_df['매출수량(lstm)']
)

full_pred_df.drop(columns=['매출여부', '매출수량(reg)', '매출수량(lstm)'], inplace=True)
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,10.794520
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),32.985085
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",8.492745
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",3.504147
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",1.251009
...,...,...,...
13505,TEST_09+7일,화담숲카페_메밀미숫가루,40.375656
13506,TEST_09+7일,화담숲카페_아메리카노 HOT,29.454053
13507,TEST_09+7일,화담숲카페_아메리카노 ICE,99.645001
13508,TEST_09+7일,화담숲카페_카페라떼 ICE,23.086480


In [68]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량'].astype(float)
    ))

    final_df = sample_submission.copy()

    menu_cols = final_df.columns[1:]
    final_df[menu_cols] = final_df[menu_cols].astype(float)

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [77]:
full_pred_df_notzero = full_pred_df.copy()
full_pred_df_notzero.loc[full_pred_df_notzero['매출수량'].abs() < 1e-5, '매출수량'] = 1

sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_hybrid = convert_to_submission_format(full_pred_df_notzero, sample_submission)
final_hybrid.to_csv('baseline_submission_hybrid.csv', index=False, encoding='utf-8-sig')

##### 다른 제출 시도들

In [ ]:
df_lstm_plz = df_lstm.copy()
df_lstm_plz['영업장명_메뉴명'] = df_lstm['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
df_lstm_end = df_lstm_plz.rename(columns = {'매출수량(lstm)' : '매출수량'})

# 1 변환
df_lstm_end.loc[df_lstm_end['매출수량'].abs() < 1e-9, '매출수량'] = 1

sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_lstm = convert_to_submission_format(df_lstm_end, sample_submission)
final_lstm.to_csv('baseline_submission_lstm.csv', index=False, encoding='utf-8-sig')

In [ ]:
full_pred_df_notzero = full_pred_df.copy()
full_pred_df_notzero.loc[full_pred_df_notzero['매출수량'].abs() < 1e-9, '매출수량'] = 1

In [ ]:
df_reg_notzero = df_reg.copy()
df_reg_notzero.loc[df_reg_notzero['매출수량'].abs() < 1e-9, '매출수량'] = 1

sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_hybrid = convert_to_submission_format(df_reg, sample_submission)
final_hybrid.to_csv('baseline_submission_reg.csv', index=False, encoding='utf-8-sig')

In [ ]:
sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_hybrid = convert_to_submission_format(full_pred_df_notzero, sample_submission)
final_hybrid.to_csv('baseline_submission_hybrid_notzero.csv', index=False, encoding='utf-8-sig')